# <font color='#fff200'> **Notebook Purpose and Introduction**

### **Purpose**
To create the Randomized Music Dataset, the Training Dataset, and the Testing Dataset which is then organized into a single CSV File.  

### **Introduction**
Combining all datasets into a single CSV file helps in organizing the data in a structured manner. This makes it easier to manage and maintain the datasets, reducing the risk of data duplication or inconsistency.

Splitting the dataset into training and testing sets is crucial for evaluating the performance of machine learning models. The training set is used to train the model, while the testing set is used to assess its performance on unseen data. This helps in estimating how well the model generalizes to new, unseen music samples.

# **Necessary Libraries and Installations**

In [ ]:
!pip install mido

In [ ]:
!pip install midi2audio

In [ ]:
!pip install pydub

In [ ]:
!sudo apt-get install fluidsynth

In [ ]:
!pip install python_speech_features

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import sys
import music21
import soundfile as sf
import csv
import scipy.io.wavfile as wav
import pickle
import random
import operator
import math
import shutil
import subprocess
import time

from librosa import midi_to_hz
from midi2audio import FluidSynth
from mido import MidiFile, MidiTrack, tempo2bpm
from pydub import AudioSegment
from music21 import converter, stream, meter
from fractions import Fraction
from scipy.signal import find_peaks
from python_speech_features import mfcc
from tempfile import TemporaryFile
from collections import defaultdict

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

# **Random Music Set Creation**


In [ ]:
iterations = 150

for i in range(iterations):
  print("Iteration:", i)
  print()
  !python /content/drive/Shareddrives/neotyagi-dataset/random-notes/generatemono.py 15
  print()
  print('=' * 100)
  print()
  time.sleep(1)

Iteration: 0

Generation for /content/drive/Shareddrives/neotyagi-dataset/GarbageSetMono/randomMONOwav000.wav complete.

Input #0, wav, from '/tmp/tmpuz28ia6q.wav':
  Duration: 00:00:28.31, bitrate: 1411 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 44100 Hz, 2 channels, s16, 1411 kb/s
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default
SDL_OpenAudio (2 channels,

In [ ]:
iterations = 10

for i in range(iterations):
  print("Iteration:", i)
  print()
  !python /content/drive/Shareddrives/neotyagi-dataset/random-notes/generatepoly.py 5
  print()
  print('=' * 100)
  print()
  time.sleep(3)

Iteration: 0


Generation for /content/drive/Shareddrives/neotyagi-dataset/GarbageSetPoly/randomPOLYwav143.wav complete.

Input #0, wav, from '/tmp/tmpbqj5bgyz.wav':
  Duration: 00:00:24.46, bitrate: 1411 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 44100 Hz, 2 channels, s16, 1411 kb/s
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default
SDL_OpenAudio (2 channels

# **Training/Testing Set Creation**


In [ ]:
def truncateMidiToDurationForMaestro(inputFile, outputFile, duration = 30):
  oldMidi = MidiFile(inputFile)
  newMidi = MidiFile(ticks_per_beat = oldMidi.ticks_per_beat)
  newTrack = MidiTrack()

  ticksPerSec = oldMidi.ticks_per_beat * tempo2bpm(oldMidi.tracks[0][0].tempo) / 60 # Calculate the ticks per second based on the tempo of the MIDI file
  desiredTicks = int(ticksPerSec * duration)                                    # Calculate the desired ticks to achieve the specified duration

  currentTicks = 0
  for track in oldMidi.tracks:
      newTrack = MidiTrack()
      currentTicks = 0
      for msg in track:
          currentTicks += msg.time
          if currentTicks > desiredTicks:

              timeRemaining = desiredTicks - (currentTicks - msg.time)          # Calculate the time remaining to reach the desired duration
              msg.time = timeRemaining                                          # Adjust the time of the current message to fit the desired duration
              newTrack.append(msg)
              break
          newTrack.append(msg)
      newMidi.tracks.append(newTrack)
  newMidi.save(outputFile)

def truncateMidiToDurationForNottingham(inputFile, outputFile, duration = 30):
    oldMidi = MidiFile(inputFile)
    newMidi = MidiFile(ticks_per_beat=oldMidi.ticks_per_beat)
    newTrack = MidiTrack()

    default_tempo = 500000                                                      # Default tempo in microseconds per beat (500000 microseconds = 120 BPM) - in case tempo information is not available.
    ticksPerSec = oldMidi.ticks_per_beat * tempo2bpm(default_tempo) / 60

    desiredTicks = int(ticksPerSec * duration)
    currentTicks = 0

    for track in oldMidi.tracks:
        newTrack = MidiTrack()
        for msg in track:
            currentTicks += msg.time
            if currentTicks > desiredTicks:
                break
            newTrack.append(msg)
        newMidi.tracks.append(newTrack)

    newMidi.save(outputFile)

################################################################################

# Truncating Source Directory
sourceDirectory = "/content/drive/Shareddrives/neotyagi-dataset/nottingham-dataset-master/MIDI/melody"
duration = 30
for filename in os.listdir(sourceDirectory):
    if "maestro" in sourceDirectory:
      if filename.endswith(".midi") and not filename.endswith("TRUNCATED.midi"):
        inputFilePath = os.path.join(sourceDirectory, filename)
        outputFilePath = os.path.join(sourceDirectory, filename[:-5] + "TRUNCATED.midi")
        truncateMidiToDurationForMaestro(inputFilePath, outputFilePath, duration)
    else:
      if filename.endswith(".mid") and not filename.endswith("TRUNCATED.mid"):
        inputFilePath = os.path.join(sourceDirectory, filename)
        outputFilePath = os.path.join(sourceDirectory, filename[:-4] + "TRUNCATED.mid")
        truncateMidiToDurationForNottingham(inputFilePath, outputFilePath, duration)
print(f"Files truncated successfully.")

In [ ]:
# Translating MIDI to WAV

sourceDirectory = "/content/drive/Shareddrives/neotyagi-dataset/nottingham-dataset-master/MIDI/melody"
for filename in os.listdir(sourceDirectory):
  if ("maestro" in sourceDirectory):
    if filename.endswith("TRUNCATED.midi"):
      filepath = os.path.join(sourceDirectory, filename)
      filepathWAV = filepath[:-5] + '.wav'
      fs = FluidSynth()
      fs.midi_to_audio(filepath, filepathWAV)
      print(f"Translated {filename} to WAV successfully.")
  else:
    if filename.endswith("TRUNCATED.mid"):
      filepath = os.path.join(sourceDirectory, filename)
      filepathWAV = filepath[:-4]+'.wav'
      fs = FluidSynth()
      fs.midi_to_audio(filepath, filepathWAV)
      print(f"Translated {filename} to WAV successfully.")

In [ ]:
# Organizing TRUNCATED.wav Files

sourceDirectory = "/content/drive/Shareddrives/neotyagi-dataset/nottingham-dataset-master/MIDI/melody"
destinationDirectory = "/content/drive/Shareddrives/neotyagi-dataset/nottingham-dataset-master/MIDI/NottinghamSets"
flagToMove = "TRUNCATED.wav"
filesMovedCount = 0

os.makedirs(destinationDirectory, exist_ok = True)                              # Create the destination directory if it does not exist

for filename in os.listdir(sourceDirectory):
    if filename.endswith(flagToMove):
        sourcePath = os.path.join(sourceDirectory, filename)
        destinationPath = os.path.join(destinationDirectory, filename)
        shutil.move(sourcePath, destinationPath)
        filesMovedCount += 1
print(f"{filesMovedCount} files have been moved.")

983 files have been moved.


# **Data to CSV**


In [ ]:
fileDataList = []
fileNumber = 1
csvTest = "/content/drive/Shareddrives/neotyagi-dataset/maestro-v3.0.0/MaestroTest/[7] CSV Test"
# csvMonophonicTrain = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/NottinghamMonoTrain[100]"
# csvPolyphonicTrain = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/MaestroPolyTrain[100]"
# csvCombinedTrain = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/CombinedTrain[100]"
# csvMonophonicTest = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/NottinghamMonoTest[50]"
# csvPolyphonicTest = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/MaestroPolyTest[50]"
# csvCombinedTest = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/CombinedTest[52]"

# Criteria 1 Constants and Definition ==========================================

noteInTuneTolerance = 7
pieceInTuneThreshold = 85
noteFrequencies = [
    65.41, 69.30, 73.42, 77.78, 82.41, 87.31, 92.50, 98.00, 103.83, 110.00, 116.54,
    123.47, 130.81, 138.59, 146.83, 155.56, 164.81, 174.61, 185.00, 196.00, 207.65,
    220.00, 233.08, 246.94, 261.63, 277.18, 293.66, 311.13, 329.63, 349.23, 369.99,
    392.00, 415.30, 440.00, 466.16, 493.88, 523.25, 554.37, 587.33, 622.25, 659.26,
    698.46, 739.99, 783.99, 830.61, 880.00, 932.33, 987.77, 1046.50, 1108.73, 1174.66,
    1244.51, 1318.51, 1396.91, 1479.98, 1567.98, 1661.22, 1760.00, 1864.66, 1975.53
]

def findClosestNoteFrequency(frequency):
    return min(noteFrequencies, key = lambda x: abs(x - frequency))

# Criteria 2 Definition ========================================================

def hasRandomNotes(audioFile, maxStabilityThreshold = 0.07, maxNoteRepetitions = 5, maxNoteDistributionStability = 14):
    audioData, sampleRate = librosa.load(audioFile)

    cqt = librosa.cqt(y = audioData, sr = sampleRate)                            # Calculate the constant-Q chromagram [a representation of an audio signal that captures the energy distribution of different pitches over time].
    chromagram = librosa.amplitude_to_db(np.abs(cqt), ref=np.max)               # Convert to decibels for easier analysis.

    # Calculating the note distribution stability
    noteDistributionStability = np.std(chromagram, axis=0).mean()

    onsetStrength = librosa.onset.onset_strength(y = audioData, sr = sampleRate) # Onset Strength: The magnitude of sudden changes in the audio signal [beginning of a note].
    tempo, _ = librosa.beat.beat_track(onset_envelope = onsetStrength, sr = sampleRate)
    _, beatTimes = librosa.beat.beat_track(onset_envelope = onsetStrength, sr = sampleRate, units = 'time') # Estimating the tempo and the beat times.

    meanTempoInterval = np.mean(np.diff(beatTimes))
    meanTempoIntervalStandardDeviation = np.std(np.diff(beatTimes))
    rhythmicStability = meanTempoIntervalStandardDeviation / meanTempoInterval

    noteOnsets = librosa.onset.onset_detect(y = audioData, sr = sampleRate)     # Detects the beginning of notes and calculating the maximum note repetitions in the audio.
    noteIntervals = np.diff(noteOnsets)
    maxNoteReps = np.max(np.bincount(noteIntervals))

    return rhythmicStability, noteDistributionStability, maxNoteReps

# Criteria 3 Definition ========================================================

def analyzeMelodicStructure(audioFile):
    audioData, samplingRate = librosa.load(audioFile)

    pitches, magnitudes = librosa.core.piptrack(y = audioData, sr = samplingRate) # Compute the pitch (fundamental frequency - lowest frequency) over time

    pitchContour = pitches[np.argmax(magnitudes, axis=0)]
    pitchContour = pitchContour.ravel()  # Flatten to a 1D Array.

    melodicIntervals = np.diff(pitchContour)                                    # Melodic intervals are the differences between consecutive pitch values.
    melodicPeaks, _ = find_peaks(pitchContour)
    melodicComplexity = np.std(melodicIntervals)                                # Measures the standard deviation of the intervals between consecutive pitches in the melodic contour.


    melodicStabilityRange = np.max(pitchContour) - np.min(pitchContour)         # Measures the span between the highest and lowest pitches in the melodic contour.
    noteDurations = np.diff(melodicPeaks) / samplingRate
    dynamicVariation = np.std(audioData)

    return melodicPeaks, melodicComplexity, melodicStabilityRange, noteDurations, dynamicVariation

# Data to CSV Implementation ===================================================

for filename in os.listdir(csvTest):
  totalNotes = 0

  if "random" in filename:
    isSignificant = False
  else:
      isSignificant = True

  if filename.endswith(".wav"):
    filepath = os.path.join(csvTest, filename)
    audioData, sampleRate = librosa.load(filepath, sr = None)
    inTuneCount = 0

    frequencyEstimatesPYIN = librosa.pyin(audioData, fmin = librosa.note_to_hz('C2'), fmax = librosa.note_to_hz('C7'), fill_na = -1)
    frequencies = frequencyEstimatesPYIN[0]
    magnitudes = frequencyEstimatesPYIN[1]


    for i in range(0, len(frequencies), 5):
      frequency = frequencies[i]
      if frequency > 0:
        closestNoteFrequency = findClosestNoteFrequency(frequency)
        if abs(frequency - closestNoteFrequency) <= noteInTuneTolerance:
            inTuneCount += 1
        totalNotes+=1

    percentageInTune = (inTuneCount / totalNotes) * 100

# End of Criteria 1 ############################################################

    rhythmStability, noteDistributionStability, maxNoteRepetitions = hasRandomNotes(filepath)

# End of Criteria 2 ############################################################

    melodicPeaks, melodicComplexity, melodicStabilityRange, noteDurations, dynamicVariation = analyzeMelodicStructure(filepath)

# End of Criteria 3 ############################################################

    fileData = {
      'Filenumber': fileNumber,
      'Filename': filename,
      '% In-Tune Notes': percentageInTune,
      'Rhythmic Stability': rhythmStability,
      'Note Distribution Stability': noteDistributionStability,
      'Melodic Stability Range': melodicStabilityRange,
      'Dynamic Variation': dynamicVariation,
      'Significant': isSignificant
    }

    fileDataList.append(fileData)
    print(f"File {fileNumber}: Finished appending data for [{filename}]\n")
    fileNumber+=1

csv_columns = [
  'Filenumber', 'Filename', '% In-Tune Notes', 'Rhythmic Stability',
  'Note Distribution Stability',
  'Melodic Stability Range', 'Dynamic Variation', 'Significant'
]

csvFileName = 'csvTest.csv'

with open(csvFileName, 'w', newline = '') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=csv_columns)

    # Write the header
    writer.writeheader()

    # Write the data for each file
    for data in fileDataList:
        writer.writerow(data)

print(f"Data has been written to {csvFileName}")

File 1: Finished appending data for [Copy of Copy of Copy of MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--1TRUNCATED.wav]

File 2: Finished appending data for [Copy of Copy of randomPOLYwav015.wav]

File 3: Finished appending data for [Copy of Copy of randomMONOwav141.wav]

Data has been written to csvTest.csv


In [ ]:
fileDataList = []
fileNumber = 1
csvMonophonicTrain = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/NottinghamMonoTrain[100]"
# csvPolyphonicTrain = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/MaestroPolyTrain[100]"
# csvCombinedTrain = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/CombinedTrain[100]"
# csvMonophonicTest = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/NottinghamMonoTest[50]"
# csvPolyphonicTest = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/MaestroPolyTest[50]"
# csvCombinedTest = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/CombinedTest[52]"

# Criteria 1 Constants and Definition ==========================================

noteInTuneTolerance = 7
pieceInTuneThreshold = 85
noteFrequencies = [
    65.41, 69.30, 73.42, 77.78, 82.41, 87.31, 92.50, 98.00, 103.83, 110.00, 116.54,
    123.47, 130.81, 138.59, 146.83, 155.56, 164.81, 174.61, 185.00, 196.00, 207.65,
    220.00, 233.08, 246.94, 261.63, 277.18, 293.66, 311.13, 329.63, 349.23, 369.99,
    392.00, 415.30, 440.00, 466.16, 493.88, 523.25, 554.37, 587.33, 622.25, 659.26,
    698.46, 739.99, 783.99, 830.61, 880.00, 932.33, 987.77, 1046.50, 1108.73, 1174.66,
    1244.51, 1318.51, 1396.91, 1479.98, 1567.98, 1661.22, 1760.00, 1864.66, 1975.53
]

def findClosestNoteFrequency(frequency):
    return min(noteFrequencies, key = lambda x: abs(x - frequency))

# Criteria 2 Definition ========================================================

def hasRandomNotes(audioFile, maxStabilityThreshold = 0.07, maxNoteRepetitions = 5, maxNoteDistributionStability = 14):
    audioData, sampleRate = librosa.load(audioFile)

    cqt = librosa.cqt(y = audioData, sr = sampleRate)                            # Calculate the constant-Q chromagram [a representation of an audio signal that captures the energy distribution of different pitches over time].
    chromagram = librosa.amplitude_to_db(np.abs(cqt), ref=np.max)               # Convert to decibels for easier analysis.

    # Calculating the note distribution stability
    noteDistributionStability = np.std(chromagram, axis=0).mean()

    onsetStrength = librosa.onset.onset_strength(y = audioData, sr = sampleRate) # Onset Strength: The magnitude of sudden changes in the audio signal [beginning of a note].
    tempo, _ = librosa.beat.beat_track(onset_envelope = onsetStrength, sr = sampleRate)
    _, beatTimes = librosa.beat.beat_track(onset_envelope = onsetStrength, sr = sampleRate, units = 'time') # Estimating the tempo and the beat times.

    meanTempoInterval = np.mean(np.diff(beatTimes))
    meanTempoIntervalStandardDeviation = np.std(np.diff(beatTimes))
    rhythmicStability = meanTempoIntervalStandardDeviation / meanTempoInterval

    noteOnsets = librosa.onset.onset_detect(y = audioData, sr = sampleRate)     # Detects the beginning of notes and calculating the maximum note repetitions in the audio.
    noteIntervals = np.diff(noteOnsets)
    maxNoteReps = np.max(np.bincount(noteIntervals))

    return rhythmicStability, noteDistributionStability, maxNoteReps

# Criteria 3 Definition ========================================================

def analyzeMelodicStructure(audioFile):
    audioData, samplingRate = librosa.load(audioFile)

    pitches, magnitudes = librosa.core.piptrack(y = audioData, sr = samplingRate) # Compute the pitch (fundamental frequency - lowest frequency) over time

    pitchContour = pitches[np.argmax(magnitudes, axis=0)]
    pitchContour = pitchContour.ravel()  # Flatten to a 1D Array.

    melodicIntervals = np.diff(pitchContour)                                    # Melodic intervals are the differences between consecutive pitch values.
    melodicPeaks, _ = find_peaks(pitchContour)
    melodicComplexity = np.std(melodicIntervals)                                # Measures the standard deviation of the intervals between consecutive pitches in the melodic contour.


    melodicStabilityRange = np.max(pitchContour) - np.min(pitchContour)         # Measures the span between the highest and lowest pitches in the melodic contour.
    noteDurations = np.diff(melodicPeaks) / samplingRate
    dynamicVariation = np.std(audioData)

    return melodicPeaks, melodicComplexity, melodicStabilityRange, noteDurations, dynamicVariation

# Data to CSV Implementation ===================================================

for filename in os.listdir(csvMonophonicTrain):
  totalNotes = 0

  if "random" in filename:
    isSignificant = False
  else:
      isSignificant = True

  if filename.endswith(".wav"):
    filepath = os.path.join(csvMonophonicTrain, filename)
    audioData, sampleRate = librosa.load(filepath, sr = None)
    inTuneCount = 0

    frequencyEstimatesPYIN = librosa.pyin(audioData, fmin = librosa.note_to_hz('C2'), fmax = librosa.note_to_hz('C7'), fill_na = -1)
    frequencies = frequencyEstimatesPYIN[0]
    magnitudes = frequencyEstimatesPYIN[1]


    for i in range(0, len(frequencies), 5):
      frequency = frequencies[i]
      if frequency > 0:
        closestNoteFrequency = findClosestNoteFrequency(frequency)
        if abs(frequency - closestNoteFrequency) <= noteInTuneTolerance:
            inTuneCount += 1
        totalNotes+=1

    percentageInTune = (inTuneCount / totalNotes) * 100

# End of Criteria 1 ############################################################

    rhythmStability, noteDistributionStability, maxNoteRepetitions = hasRandomNotes(filepath)

# End of Criteria 2 ############################################################

    melodicPeaks, melodicComplexity, melodicStabilityRange, noteDurations, dynamicVariation = analyzeMelodicStructure(filepath)

# End of Criteria 3 ############################################################

    fileData = {
      'Filenumber': fileNumber,
      'Filename': filename,
      '% In-Tune Notes': percentageInTune,
      'Rhythmic Stability': rhythmStability,
      'Note Distribution Stability': noteDistributionStability,
      'Melodic Stability Range': melodicStabilityRange,
      'Dynamic Variation': dynamicVariation,
      'Significant': isSignificant
    }

    fileDataList.append(fileData)
    print(f"File {fileNumber}: Finished appending data for [{filename}]\n")
    fileNumber+=1

csv_columns = [
  'Filenumber', 'Filename', '% In-Tune Notes', 'Rhythmic Stability',
  'Note Distribution Stability',
  'Melodic Stability Range', 'Dynamic Variation', 'Significant'
]

csvFileName = 'monoNottinghamTrain.csv'

with open(csvFileName, 'w', newline = '') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=csv_columns)

    # Write the header
    writer.writeheader()

    # Write the data for each file
    for data in fileDataList:
        writer.writerow(data)

print(f"Data has been written to {csvFileName}")

File 1: Finished appending data for [xmas12TRUNCATED.wav]

File 2: Finished appending data for [xmas6TRUNCATED.wav]

File 3: Finished appending data for [xmas5TRUNCATED.wav]

File 4: Finished appending data for [waltzes16TRUNCATED.wav]

File 5: Finished appending data for [waltzes2TRUNCATED.wav]

File 6: Finished appending data for [reelsu-z11TRUNCATED.wav]

File 7: Finished appending data for [waltzes46TRUNCATED.wav]

File 8: Finished appending data for [waltzes11TRUNCATED.wav]

File 9: Finished appending data for [waltzes49TRUNCATED.wav]

File 10: Finished appending data for [waltzes42TRUNCATED.wav]

File 11: Finished appending data for [xmas4TRUNCATED.wav]

File 12: Finished appending data for [reelsu-z9TRUNCATED.wav]

File 13: Finished appending data for [slip8TRUNCATED.wav]

File 14: Finished appending data for [reelsu-z34TRUNCATED.wav]

File 15: Finished appending data for [reelsu-z32TRUNCATED.wav]

File 16: Finished appending data for [slip7TRUNCATED.wav]

File 17: Finished appe

In [ ]:
fileDataList = []
fileNumber = 1
# csvMonophonicTrain = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/NottinghamMonoTrain[100]"
csvPolyphonicTrain = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/MaestroPolyTrain[100]"
# csvCombinedTrain = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/CombinedTrain[100]"
# csvMonophonicTest = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/NottinghamMonoTest[50]"
# csvPolyphonicTest = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/MaestroPolyTest[50]"
# csvCombinedTest = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/CombinedTest[52]"

# Criteria 1 Constants and Definition ==========================================

noteInTuneTolerance = 7
pieceInTuneThreshold = 85
noteFrequencies = [
    65.41, 69.30, 73.42, 77.78, 82.41, 87.31, 92.50, 98.00, 103.83, 110.00, 116.54,
    123.47, 130.81, 138.59, 146.83, 155.56, 164.81, 174.61, 185.00, 196.00, 207.65,
    220.00, 233.08, 246.94, 261.63, 277.18, 293.66, 311.13, 329.63, 349.23, 369.99,
    392.00, 415.30, 440.00, 466.16, 493.88, 523.25, 554.37, 587.33, 622.25, 659.26,
    698.46, 739.99, 783.99, 830.61, 880.00, 932.33, 987.77, 1046.50, 1108.73, 1174.66,
    1244.51, 1318.51, 1396.91, 1479.98, 1567.98, 1661.22, 1760.00, 1864.66, 1975.53
]

def findClosestNoteFrequency(frequency):
    return min(noteFrequencies, key = lambda x: abs(x - frequency))

# Criteria 2 Definition ========================================================

def hasRandomNotes(audioFile, maxStabilityThreshold = 0.07, maxNoteRepetitions = 5, maxNoteDistributionStability = 14):
    audioData, sampleRate = librosa.load(audioFile)

    cqt = librosa.cqt(y = audioData, sr = sampleRate)                            # Calculate the constant-Q chromagram [a representation of an audio signal that captures the energy distribution of different pitches over time].
    chromagram = librosa.amplitude_to_db(np.abs(cqt), ref=np.max)               # Convert to decibels for easier analysis.

    # Calculating the note distribution stability
    noteDistributionStability = np.std(chromagram, axis=0).mean()

    onsetStrength = librosa.onset.onset_strength(y = audioData, sr = sampleRate) # Onset Strength: The magnitude of sudden changes in the audio signal [beginning of a note].
    tempo, _ = librosa.beat.beat_track(onset_envelope = onsetStrength, sr = sampleRate)
    _, beatTimes = librosa.beat.beat_track(onset_envelope = onsetStrength, sr = sampleRate, units = 'time') # Estimating the tempo and the beat times.

    meanTempoInterval = np.mean(np.diff(beatTimes))
    meanTempoIntervalStandardDeviation = np.std(np.diff(beatTimes))
    rhythmicStability = meanTempoIntervalStandardDeviation / meanTempoInterval

    noteOnsets = librosa.onset.onset_detect(y = audioData, sr = sampleRate)     # Detects the beginning of notes and calculating the maximum note repetitions in the audio.
    noteIntervals = np.diff(noteOnsets)
    maxNoteReps = np.max(np.bincount(noteIntervals))

    return rhythmicStability, noteDistributionStability, maxNoteReps

# Criteria 3 Definition ========================================================

def analyzeMelodicStructure(audioFile):
    audioData, samplingRate = librosa.load(audioFile)

    pitches, magnitudes = librosa.core.piptrack(y = audioData, sr = samplingRate) # Compute the pitch (fundamental frequency - lowest frequency) over time

    pitchContour = pitches[np.argmax(magnitudes, axis=0)]
    pitchContour = pitchContour.ravel()  # Flatten to a 1D Array.

    melodicIntervals = np.diff(pitchContour)                                    # Melodic intervals are the differences between consecutive pitch values.
    melodicPeaks, _ = find_peaks(pitchContour)
    melodicComplexity = np.std(melodicIntervals)                                # Measures the standard deviation of the intervals between consecutive pitches in the melodic contour.


    melodicStabilityRange = np.max(pitchContour) - np.min(pitchContour)         # Measures the span between the highest and lowest pitches in the melodic contour.
    noteDurations = np.diff(melodicPeaks) / samplingRate
    dynamicVariation = np.std(audioData)

    return melodicPeaks, melodicComplexity, melodicStabilityRange, noteDurations, dynamicVariation

# Data to CSV Implementation ===================================================

for filename in os.listdir(csvPolyphonicTrain):
  totalNotes = 0

  if "random" in filename:
    isSignificant = False
  else:
      isSignificant = True

  if filename.endswith(".wav"):
    filepath = os.path.join(csvPolyphonicTrain, filename)
    audioData, sampleRate = librosa.load(filepath, sr = None)
    inTuneCount = 0

    frequencyEstimatesPYIN = librosa.pyin(audioData, fmin = librosa.note_to_hz('C2'), fmax = librosa.note_to_hz('C7'), fill_na = -1)
    frequencies = frequencyEstimatesPYIN[0]
    magnitudes = frequencyEstimatesPYIN[1]


    for i in range(0, len(frequencies), 5):
      frequency = frequencies[i]
      if frequency > 0:
        closestNoteFrequency = findClosestNoteFrequency(frequency)
        if abs(frequency - closestNoteFrequency) <= noteInTuneTolerance:
            inTuneCount += 1
        totalNotes+=1

    percentageInTune = (inTuneCount / totalNotes) * 100

# End of Criteria 1 ############################################################

    rhythmStability, noteDistributionStability, maxNoteRepetitions = hasRandomNotes(filepath)

# End of Criteria 2 ############################################################

    melodicPeaks, melodicComplexity, melodicStabilityRange, noteDurations, dynamicVariation = analyzeMelodicStructure(filepath)

# End of Criteria 3 ############################################################

    fileData = {
      'Filenumber': fileNumber,
      'Filename': filename,
      '% In-Tune Notes': percentageInTune,
      'Rhythmic Stability': rhythmStability,
      'Note Distribution Stability': noteDistributionStability,
      'Melodic Stability Range': melodicStabilityRange,
      'Dynamic Variation': dynamicVariation,
      'Significant': isSignificant
    }

    fileDataList.append(fileData)
    print(f"File {fileNumber}: Finished appending data for [{filename}]\n")
    fileNumber+=1

csv_columns = [
  'Filenumber', 'Filename', '% In-Tune Notes', 'Rhythmic Stability',
  'Note Distribution Stability',
  'Melodic Stability Range', 'Dynamic Variation', 'Significant'
]

csvFileName = 'polyMaestroTrain.csv'

with open(csvFileName, 'w', newline = '') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=csv_columns)

    # Write the header
    writer.writeheader()

    # Write the data for each file
    for data in fileDataList:
        writer.writerow(data)

print(f"Data has been written to {csvFileName}")

File 1: Finished appending data for [MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1TRUNCATED.wav]

File 2: Finished appending data for [MIDI-Unprocessed_Chamber4_MID--AUDIO_11_R3_2018_wav--1TRUNCATED.wav]

File 3: Finished appending data for [MIDI-Unprocessed_Chamber2_MID--AUDIO_09_R3_2018_wav--1TRUNCATED.wav]

File 4: Finished appending data for [MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--2TRUNCATED.wav]

File 5: Finished appending data for [MIDI-Unprocessed_Recital1-3_MID--AUDIO_02_R1_2018_wav--4TRUNCATED.wav]

File 6: Finished appending data for [MIDI-Unprocessed_Recital1-3_MID--AUDIO_02_R1_2018_wav--3TRUNCATED.wav]

File 7: Finished appending data for [MIDI-Unprocessed_Recital1-3_MID--AUDIO_03_R1_2018_wav--2TRUNCATED.wav]

File 8: Finished appending data for [MIDI-Unprocessed_Recital1-3_MID--AUDIO_03_R1_2018_wav--4TRUNCATED.wav]

File 9: Finished appending data for [MIDI-Unprocessed_Recital1-3_MID--AUDIO_01_R1_2018_wav--1TRUNCATED.wav]

File 10: Finished appending

In [ ]:
fileDataList = []
fileNumber = 1
# csvMonophonicTrain = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/NottinghamMonoTrain[100]"
# csvPolyphonicTrain = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/MaestroPolyTrain[100]"
csvCombinedTrain = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/CombinedTrain[100]"
# csvMonophonicTest = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/NottinghamMonoTest[50]"
# csvPolyphonicTest = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/MaestroPolyTest[50]"
# csvCombinedTest = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/CombinedTest[52]"

# Criteria 1 Constants and Definition ==========================================

noteInTuneTolerance = 7
pieceInTuneThreshold = 85
noteFrequencies = [
    65.41, 69.30, 73.42, 77.78, 82.41, 87.31, 92.50, 98.00, 103.83, 110.00, 116.54,
    123.47, 130.81, 138.59, 146.83, 155.56, 164.81, 174.61, 185.00, 196.00, 207.65,
    220.00, 233.08, 246.94, 261.63, 277.18, 293.66, 311.13, 329.63, 349.23, 369.99,
    392.00, 415.30, 440.00, 466.16, 493.88, 523.25, 554.37, 587.33, 622.25, 659.26,
    698.46, 739.99, 783.99, 830.61, 880.00, 932.33, 987.77, 1046.50, 1108.73, 1174.66,
    1244.51, 1318.51, 1396.91, 1479.98, 1567.98, 1661.22, 1760.00, 1864.66, 1975.53
]

def findClosestNoteFrequency(frequency):
    return min(noteFrequencies, key = lambda x: abs(x - frequency))

# Criteria 2 Definition ========================================================

def hasRandomNotes(audioFile, maxStabilityThreshold = 0.07, maxNoteRepetitions = 5, maxNoteDistributionStability = 14):
    audioData, sampleRate = librosa.load(audioFile)

    cqt = librosa.cqt(y = audioData, sr = sampleRate)                            # Calculate the constant-Q chromagram [a representation of an audio signal that captures the energy distribution of different pitches over time].
    chromagram = librosa.amplitude_to_db(np.abs(cqt), ref=np.max)               # Convert to decibels for easier analysis.

    # Calculating the note distribution stability
    noteDistributionStability = np.std(chromagram, axis=0).mean()

    onsetStrength = librosa.onset.onset_strength(y = audioData, sr = sampleRate) # Onset Strength: The magnitude of sudden changes in the audio signal [beginning of a note].
    tempo, _ = librosa.beat.beat_track(onset_envelope = onsetStrength, sr = sampleRate)
    _, beatTimes = librosa.beat.beat_track(onset_envelope = onsetStrength, sr = sampleRate, units = 'time') # Estimating the tempo and the beat times.

    meanTempoInterval = np.mean(np.diff(beatTimes))
    meanTempoIntervalStandardDeviation = np.std(np.diff(beatTimes))
    rhythmicStability = meanTempoIntervalStandardDeviation / meanTempoInterval

    noteOnsets = librosa.onset.onset_detect(y = audioData, sr = sampleRate)     # Detects the beginning of notes and calculating the maximum note repetitions in the audio.
    noteIntervals = np.diff(noteOnsets)
    maxNoteReps = np.max(np.bincount(noteIntervals))

    return rhythmicStability, noteDistributionStability, maxNoteReps

# Criteria 3 Definition ========================================================

def analyzeMelodicStructure(audioFile):
    audioData, samplingRate = librosa.load(audioFile)

    pitches, magnitudes = librosa.core.piptrack(y = audioData, sr = samplingRate) # Compute the pitch (fundamental frequency - lowest frequency) over time

    pitchContour = pitches[np.argmax(magnitudes, axis=0)]
    pitchContour = pitchContour.ravel()  # Flatten to a 1D Array.

    melodicIntervals = np.diff(pitchContour)                                    # Melodic intervals are the differences between consecutive pitch values.
    melodicPeaks, _ = find_peaks(pitchContour)
    melodicComplexity = np.std(melodicIntervals)                                # Measures the standard deviation of the intervals between consecutive pitches in the melodic contour.


    melodicStabilityRange = np.max(pitchContour) - np.min(pitchContour)         # Measures the span between the highest and lowest pitches in the melodic contour.
    noteDurations = np.diff(melodicPeaks) / samplingRate
    dynamicVariation = np.std(audioData)

    return melodicPeaks, melodicComplexity, melodicStabilityRange, noteDurations, dynamicVariation

# Data to CSV Implementation ===================================================

for filename in os.listdir(csvCombinedTrain):
  totalNotes = 0

  if "random" in filename:
    isSignificant = False
  else:
      isSignificant = True

  if filename.endswith(".wav"):
    filepath = os.path.join(csvCombinedTrain, filename)
    audioData, sampleRate = librosa.load(filepath, sr = None)
    inTuneCount = 0

    frequencyEstimatesPYIN = librosa.pyin(audioData, fmin = librosa.note_to_hz('C2'), fmax = librosa.note_to_hz('C7'), fill_na = -1)
    frequencies = frequencyEstimatesPYIN[0]
    magnitudes = frequencyEstimatesPYIN[1]


    for i in range(0, len(frequencies), 5):
      frequency = frequencies[i]
      if frequency > 0:
        closestNoteFrequency = findClosestNoteFrequency(frequency)
        if abs(frequency - closestNoteFrequency) <= noteInTuneTolerance:
            inTuneCount += 1
        totalNotes+=1

    percentageInTune = (inTuneCount / totalNotes) * 100

# End of Criteria 1 ############################################################

    rhythmStability, noteDistributionStability, maxNoteRepetitions = hasRandomNotes(filepath)

# End of Criteria 2 ############################################################

    melodicPeaks, melodicComplexity, melodicStabilityRange, noteDurations, dynamicVariation = analyzeMelodicStructure(filepath)

# End of Criteria 3 ############################################################

    fileData = {
      'Filenumber': fileNumber,
      'Filename': filename,
      '% In-Tune Notes': percentageInTune,
      'Rhythmic Stability': rhythmStability,
      'Note Distribution Stability': noteDistributionStability,
      'Melodic Stability Range': melodicStabilityRange,
      'Dynamic Variation': dynamicVariation,
      'Significant': isSignificant
    }

    fileDataList.append(fileData)
    print(f"File {fileNumber}: Finished appending data for [{filename}]\n")
    fileNumber+=1

csv_columns = [
  'Filenumber', 'Filename', '% In-Tune Notes', 'Rhythmic Stability',
  'Note Distribution Stability',
  'Melodic Stability Range', 'Dynamic Variation', 'Significant'
]

csvFileName = 'combinedTrain.csv'

with open(csvFileName, 'w', newline = '') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=csv_columns)

    # Write the header
    writer.writeheader()

    # Write the data for each file
    for data in fileDataList:
        writer.writerow(data)

print(f"Data has been written to {csvFileName}")

File 1: Finished appending data for [Copy of playford1TRUNCATED.wav]

File 2: Finished appending data for [Copy of morris29TRUNCATED.wav]

File 3: Finished appending data for [Copy of morris30TRUNCATED.wav]

File 4: Finished appending data for [Copy of playford7TRUNCATED.wav]

File 5: Finished appending data for [Copy of playford6TRUNCATED.wav]

File 6: Finished appending data for [Copy of morris31TRUNCATED.wav]

File 7: Finished appending data for [Copy of playford8TRUNCATED.wav]

File 8: Finished appending data for [Copy of playford4TRUNCATED.wav]

File 9: Finished appending data for [Copy of playford3TRUNCATED.wav]

File 10: Finished appending data for [Copy of playford2TRUNCATED.wav]

File 11: Finished appending data for [Copy of playford13TRUNCATED.wav]

File 12: Finished appending data for [Copy of playford10TRUNCATED.wav]

File 13: Finished appending data for [Copy of playford14TRUNCATED.wav]

File 14: Finished appending data for [Copy of playford9TRUNCATED.wav]

File 15: Finish

In [ ]:
fileDataList = []
fileNumber = 1
# csvMonophonicTrain = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/NottinghamMonoTrain[100]"
# csvPolyphonicTrain = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/MaestroPolyTrain[100]"
# csvCombinedTrain = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/CombinedTrain[100]"
csvMonophonicTest = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/NottinghamMonoTest[50]"
# csvPolyphonicTest = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/MaestroPolyTest[50]"
# csvCombinedTest = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/CombinedTest[52]"

# Criteria 1 Constants and Definition ==========================================

noteInTuneTolerance = 7
pieceInTuneThreshold = 85
noteFrequencies = [
    65.41, 69.30, 73.42, 77.78, 82.41, 87.31, 92.50, 98.00, 103.83, 110.00, 116.54,
    123.47, 130.81, 138.59, 146.83, 155.56, 164.81, 174.61, 185.00, 196.00, 207.65,
    220.00, 233.08, 246.94, 261.63, 277.18, 293.66, 311.13, 329.63, 349.23, 369.99,
    392.00, 415.30, 440.00, 466.16, 493.88, 523.25, 554.37, 587.33, 622.25, 659.26,
    698.46, 739.99, 783.99, 830.61, 880.00, 932.33, 987.77, 1046.50, 1108.73, 1174.66,
    1244.51, 1318.51, 1396.91, 1479.98, 1567.98, 1661.22, 1760.00, 1864.66, 1975.53
]

def findClosestNoteFrequency(frequency):
    return min(noteFrequencies, key = lambda x: abs(x - frequency))

# Criteria 2 Definition ========================================================

def hasRandomNotes(audioFile, maxStabilityThreshold = 0.07, maxNoteRepetitions = 5, maxNoteDistributionStability = 14):
    audioData, sampleRate = librosa.load(audioFile)

    cqt = librosa.cqt(y = audioData, sr = sampleRate)                            # Calculate the constant-Q chromagram [a representation of an audio signal that captures the energy distribution of different pitches over time].
    chromagram = librosa.amplitude_to_db(np.abs(cqt), ref=np.max)               # Convert to decibels for easier analysis.

    # Calculating the note distribution stability
    noteDistributionStability = np.std(chromagram, axis=0).mean()

    onsetStrength = librosa.onset.onset_strength(y = audioData, sr = sampleRate) # Onset Strength: The magnitude of sudden changes in the audio signal [beginning of a note].
    tempo, _ = librosa.beat.beat_track(onset_envelope = onsetStrength, sr = sampleRate)
    _, beatTimes = librosa.beat.beat_track(onset_envelope = onsetStrength, sr = sampleRate, units = 'time') # Estimating the tempo and the beat times.

    meanTempoInterval = np.mean(np.diff(beatTimes))
    meanTempoIntervalStandardDeviation = np.std(np.diff(beatTimes))
    rhythmicStability = meanTempoIntervalStandardDeviation / meanTempoInterval

    noteOnsets = librosa.onset.onset_detect(y = audioData, sr = sampleRate)     # Detects the beginning of notes and calculating the maximum note repetitions in the audio.
    noteIntervals = np.diff(noteOnsets)
    maxNoteReps = np.max(np.bincount(noteIntervals))

    return rhythmicStability, noteDistributionStability, maxNoteReps

# Criteria 3 Definition ========================================================

def analyzeMelodicStructure(audioFile):
    audioData, samplingRate = librosa.load(audioFile)

    pitches, magnitudes = librosa.core.piptrack(y = audioData, sr = samplingRate) # Compute the pitch (fundamental frequency - lowest frequency) over time

    pitchContour = pitches[np.argmax(magnitudes, axis=0)]
    pitchContour = pitchContour.ravel()  # Flatten to a 1D Array.

    melodicIntervals = np.diff(pitchContour)                                    # Melodic intervals are the differences between consecutive pitch values.
    melodicPeaks, _ = find_peaks(pitchContour)
    melodicComplexity = np.std(melodicIntervals)                                # Measures the standard deviation of the intervals between consecutive pitches in the melodic contour.


    melodicStabilityRange = np.max(pitchContour) - np.min(pitchContour)         # Measures the span between the highest and lowest pitches in the melodic contour.
    noteDurations = np.diff(melodicPeaks) / samplingRate
    dynamicVariation = np.std(audioData)

    return melodicPeaks, melodicComplexity, melodicStabilityRange, noteDurations, dynamicVariation

# Data to CSV Implementation ===================================================

for filename in os.listdir(csvMonophonicTest):
  totalNotes = 0

  if "random" in filename:
    isSignificant = False
  else:
      isSignificant = True

  if filename.endswith(".wav"):
    filepath = os.path.join(csvMonophonicTest, filename)
    audioData, sampleRate = librosa.load(filepath, sr = None)
    inTuneCount = 0

    frequencyEstimatesPYIN = librosa.pyin(audioData, fmin = librosa.note_to_hz('C2'), fmax = librosa.note_to_hz('C7'), fill_na = -1)
    frequencies = frequencyEstimatesPYIN[0]
    magnitudes = frequencyEstimatesPYIN[1]


    for i in range(0, len(frequencies), 5):
      frequency = frequencies[i]
      if frequency > 0:
        closestNoteFrequency = findClosestNoteFrequency(frequency)
        if abs(frequency - closestNoteFrequency) <= noteInTuneTolerance:
            inTuneCount += 1
        totalNotes+=1

    percentageInTune = (inTuneCount / totalNotes) * 100

# End of Criteria 1 ############################################################

    rhythmStability, noteDistributionStability, maxNoteRepetitions = hasRandomNotes(filepath)

# End of Criteria 2 ############################################################

    melodicPeaks, melodicComplexity, melodicStabilityRange, noteDurations, dynamicVariation = analyzeMelodicStructure(filepath)

# End of Criteria 3 ############################################################

    fileData = {
      'Filenumber': fileNumber,
      'Filename': filename,
      '% In-Tune Notes': percentageInTune,
      'Rhythmic Stability': rhythmStability,
      'Note Distribution Stability': noteDistributionStability,
      'Melodic Stability Range': melodicStabilityRange,
      'Dynamic Variation': dynamicVariation,
      'Significant': isSignificant
    }

    fileDataList.append(fileData)
    print(f"File {fileNumber}: Finished appending data for [{filename}]\n")
    fileNumber+=1

csv_columns = [
  'Filenumber', 'Filename', '% In-Tune Notes', 'Rhythmic Stability',
  'Note Distribution Stability',
  'Melodic Stability Range', 'Dynamic Variation', 'Significant'
]

csvFileName = 'monoNottinghamTest.csv'

with open(csvFileName, 'w', newline = '') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=csv_columns)

    # Write the header
    writer.writeheader()

    # Write the data for each file
    for data in fileDataList:
        writer.writerow(data)

print(f"Data has been written to {csvFileName}")

File 1: Finished appending data for [reelsr-t62TRUNCATED.wav]

File 2: Finished appending data for [reelsr-t49TRUNCATED.wav]

File 3: Finished appending data for [reelsr-t70TRUNCATED.wav]

File 4: Finished appending data for [reelsr-t64TRUNCATED.wav]

File 5: Finished appending data for [reelsr-t66TRUNCATED.wav]

File 6: Finished appending data for [reelsr-t67TRUNCATED.wav]

File 7: Finished appending data for [reelsr-t48TRUNCATED.wav]

File 8: Finished appending data for [reelsr-t57TRUNCATED.wav]

File 9: Finished appending data for [reelsr-t47TRUNCATED.wav]

File 10: Finished appending data for [reelsr-t68TRUNCATED.wav]

File 11: Finished appending data for [reelsr-t69TRUNCATED.wav]

File 12: Finished appending data for [reelsr-t59TRUNCATED.wav]

File 13: Finished appending data for [reelsr-t52TRUNCATED.wav]

File 14: Finished appending data for [reelsr-t58TRUNCATED.wav]

File 15: Finished appending data for [reelsr-t50TRUNCATED.wav]

File 16: Finished appending data for [reelsr-t61T

In [ ]:
fileDataList = []
fileNumber = 1
# csvMonophonicTrain = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/NottinghamMonoTrain[100]"
# csvPolyphonicTrain = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/MaestroPolyTrain[100]"
# csvCombinedTrain = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/CombinedTrain[100]"
# csvMonophonicTest = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/NottinghamMonoTest[50]"
csvPolyphonicTest = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/MaestroPolyTest[50]"
# csvCombinedTest = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/CombinedTest[52]"

# Criteria 1 Constants and Definition ==========================================

noteInTuneTolerance = 7
pieceInTuneThreshold = 85
noteFrequencies = [
    65.41, 69.30, 73.42, 77.78, 82.41, 87.31, 92.50, 98.00, 103.83, 110.00, 116.54,
    123.47, 130.81, 138.59, 146.83, 155.56, 164.81, 174.61, 185.00, 196.00, 207.65,
    220.00, 233.08, 246.94, 261.63, 277.18, 293.66, 311.13, 329.63, 349.23, 369.99,
    392.00, 415.30, 440.00, 466.16, 493.88, 523.25, 554.37, 587.33, 622.25, 659.26,
    698.46, 739.99, 783.99, 830.61, 880.00, 932.33, 987.77, 1046.50, 1108.73, 1174.66,
    1244.51, 1318.51, 1396.91, 1479.98, 1567.98, 1661.22, 1760.00, 1864.66, 1975.53
]

def findClosestNoteFrequency(frequency):
    return min(noteFrequencies, key = lambda x: abs(x - frequency))

# Criteria 2 Definition ========================================================

def hasRandomNotes(audioFile, maxStabilityThreshold = 0.07, maxNoteRepetitions = 5, maxNoteDistributionStability = 14):
    audioData, sampleRate = librosa.load(audioFile)

    cqt = librosa.cqt(y = audioData, sr = sampleRate)                            # Calculate the constant-Q chromagram [a representation of an audio signal that captures the energy distribution of different pitches over time].
    chromagram = librosa.amplitude_to_db(np.abs(cqt), ref=np.max)               # Convert to decibels for easier analysis.

    # Calculating the note distribution stability
    noteDistributionStability = np.std(chromagram, axis=0).mean()

    onsetStrength = librosa.onset.onset_strength(y = audioData, sr = sampleRate) # Onset Strength: The magnitude of sudden changes in the audio signal [beginning of a note].
    tempo, _ = librosa.beat.beat_track(onset_envelope = onsetStrength, sr = sampleRate)
    _, beatTimes = librosa.beat.beat_track(onset_envelope = onsetStrength, sr = sampleRate, units = 'time') # Estimating the tempo and the beat times.

    meanTempoInterval = np.mean(np.diff(beatTimes))
    meanTempoIntervalStandardDeviation = np.std(np.diff(beatTimes))
    rhythmicStability = meanTempoIntervalStandardDeviation / meanTempoInterval

    noteOnsets = librosa.onset.onset_detect(y = audioData, sr = sampleRate)     # Detects the beginning of notes and calculating the maximum note repetitions in the audio.
    noteIntervals = np.diff(noteOnsets)
    maxNoteReps = np.max(np.bincount(noteIntervals))

    return rhythmicStability, noteDistributionStability, maxNoteReps

# Criteria 3 Definition ========================================================

def analyzeMelodicStructure(audioFile):
    audioData, samplingRate = librosa.load(audioFile)

    pitches, magnitudes = librosa.core.piptrack(y = audioData, sr = samplingRate) # Compute the pitch (fundamental frequency - lowest frequency) over time

    pitchContour = pitches[np.argmax(magnitudes, axis=0)]
    pitchContour = pitchContour.ravel()  # Flatten to a 1D Array.

    melodicIntervals = np.diff(pitchContour)                                    # Melodic intervals are the differences between consecutive pitch values.
    melodicPeaks, _ = find_peaks(pitchContour)
    melodicComplexity = np.std(melodicIntervals)                                # Measures the standard deviation of the intervals between consecutive pitches in the melodic contour.


    melodicStabilityRange = np.max(pitchContour) - np.min(pitchContour)         # Measures the span between the highest and lowest pitches in the melodic contour.
    noteDurations = np.diff(melodicPeaks) / samplingRate
    dynamicVariation = np.std(audioData)

    return melodicPeaks, melodicComplexity, melodicStabilityRange, noteDurations, dynamicVariation

# Data to CSV Implementation ===================================================

for filename in os.listdir(csvPolyphonicTest):
  totalNotes = 0

  if "random" in filename:
    isSignificant = False
  else:
      isSignificant = True

  if filename.endswith(".wav"):
    filepath = os.path.join(csvPolyphonicTest, filename)
    audioData, sampleRate = librosa.load(filepath, sr = None)
    inTuneCount = 0

    frequencyEstimatesPYIN = librosa.pyin(audioData, fmin = librosa.note_to_hz('C2'), fmax = librosa.note_to_hz('C7'), fill_na = -1)
    frequencies = frequencyEstimatesPYIN[0]
    magnitudes = frequencyEstimatesPYIN[1]


    for i in range(0, len(frequencies), 5):
      frequency = frequencies[i]
      if frequency > 0:
        closestNoteFrequency = findClosestNoteFrequency(frequency)
        if abs(frequency - closestNoteFrequency) <= noteInTuneTolerance:
            inTuneCount += 1
        totalNotes+=1

    percentageInTune = (inTuneCount / totalNotes) * 100

# End of Criteria 1 ############################################################

    rhythmStability, noteDistributionStability, maxNoteRepetitions = hasRandomNotes(filepath)

# End of Criteria 2 ############################################################

    melodicPeaks, melodicComplexity, melodicStabilityRange, noteDurations, dynamicVariation = analyzeMelodicStructure(filepath)

# End of Criteria 3 ############################################################

    fileData = {
      'Filenumber': fileNumber,
      'Filename': filename,
      '% In-Tune Notes': percentageInTune,
      'Rhythmic Stability': rhythmStability,
      'Note Distribution Stability': noteDistributionStability,
      'Melodic Stability Range': melodicStabilityRange,
      'Dynamic Variation': dynamicVariation,
      'Significant': isSignificant
    }

    fileDataList.append(fileData)
    print(f"File {fileNumber}: Finished appending data for [{filename}]\n")
    fileNumber+=1

csv_columns = [
  'Filenumber', 'Filename', '% In-Tune Notes', 'Rhythmic Stability',
  'Note Distribution Stability',
  'Melodic Stability Range', 'Dynamic Variation', 'Significant'
]

csvFileName = 'polyMaestroTest.csv'

with open(csvFileName, 'w', newline = '') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=csv_columns)

    # Write the header
    writer.writeheader()

    # Write the data for each file
    for data in fileDataList:
        writer.writerow(data)

print(f"Data has been written to {csvFileName}")

File 1: Finished appending data for [MIDI-Unprocessed_049_PIANO049_MID--AUDIO-split_07-06-17_Piano-e_2-06_wav--2TRUNCATED.wav]

File 2: Finished appending data for [MIDI-Unprocessed_046_PIANO046_MID--AUDIO-split_07-06-17_Piano-e_2-02_wav--4TRUNCATED.wav]

File 3: Finished appending data for [MIDI-Unprocessed_045_PIANO045_MID--AUDIO-split_07-06-17_Piano-e_2-01_wav--4TRUNCATED.wav]

File 4: Finished appending data for [MIDI-Unprocessed_044_PIANO044_MID--AUDIO-split_07-06-17_Piano-e_1-04_wav--3TRUNCATED.wav]

File 5: Finished appending data for [MIDI-Unprocessed_044_PIANO044_MID--AUDIO-split_07-06-17_Piano-e_1-04_wav--2TRUNCATED.wav]

File 6: Finished appending data for [MIDI-Unprocessed_043_PIANO043_MID--AUDIO-split_07-06-17_Piano-e_1-03_wav--4TRUNCATED.wav]

File 7: Finished appending data for [MIDI-Unprocessed_048_PIANO048_MID--AUDIO-split_07-06-17_Piano-e_2-05_wav--1TRUNCATED.wav]

File 8: Finished appending data for [MIDI-Unprocessed_043_PIANO043_MID--AUDIO-split_07-06-17_Piano-e_1-0

In [ ]:
fileDataList = []
fileNumber = 1
# csvMonophonicTrain = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/NottinghamMonoTrain[100]"
# csvPolyphonicTrain = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/MaestroPolyTrain[100]"
# csvCombinedTrain = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/CombinedTrain[100]"
# csvMonophonicTest = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/NottinghamMonoTest[50]"
# csvPolyphonicTest = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/MaestroPolyTest[50]"
csvCombinedTest = "/content/drive/Shareddrives/neotyagi-dataset/CSV Creation/CombinedTest[52]"

# Criteria 1 Constants and Definition ==========================================

noteInTuneTolerance = 7
pieceInTuneThreshold = 85
noteFrequencies = [
    65.41, 69.30, 73.42, 77.78, 82.41, 87.31, 92.50, 98.00, 103.83, 110.00, 116.54,
    123.47, 130.81, 138.59, 146.83, 155.56, 164.81, 174.61, 185.00, 196.00, 207.65,
    220.00, 233.08, 246.94, 261.63, 277.18, 293.66, 311.13, 329.63, 349.23, 369.99,
    392.00, 415.30, 440.00, 466.16, 493.88, 523.25, 554.37, 587.33, 622.25, 659.26,
    698.46, 739.99, 783.99, 830.61, 880.00, 932.33, 987.77, 1046.50, 1108.73, 1174.66,
    1244.51, 1318.51, 1396.91, 1479.98, 1567.98, 1661.22, 1760.00, 1864.66, 1975.53
]

def findClosestNoteFrequency(frequency):
    return min(noteFrequencies, key = lambda x: abs(x - frequency))

# Criteria 2 Definition ========================================================

def hasRandomNotes(audioFile, maxStabilityThreshold = 0.07, maxNoteRepetitions = 5, maxNoteDistributionStability = 14):
    audioData, sampleRate = librosa.load(audioFile)

    cqt = librosa.cqt(y = audioData, sr = sampleRate)                            # Calculate the constant-Q chromagram [a representation of an audio signal that captures the energy distribution of different pitches over time].
    chromagram = librosa.amplitude_to_db(np.abs(cqt), ref=np.max)               # Convert to decibels for easier analysis.

    # Calculating the note distribution stability
    noteDistributionStability = np.std(chromagram, axis=0).mean()

    onsetStrength = librosa.onset.onset_strength(y = audioData, sr = sampleRate) # Onset Strength: The magnitude of sudden changes in the audio signal [beginning of a note].
    tempo, _ = librosa.beat.beat_track(onset_envelope = onsetStrength, sr = sampleRate)
    _, beatTimes = librosa.beat.beat_track(onset_envelope = onsetStrength, sr = sampleRate, units = 'time') # Estimating the tempo and the beat times.

    meanTempoInterval = np.mean(np.diff(beatTimes))
    meanTempoIntervalStandardDeviation = np.std(np.diff(beatTimes))
    rhythmicStability = meanTempoIntervalStandardDeviation / meanTempoInterval

    noteOnsets = librosa.onset.onset_detect(y = audioData, sr = sampleRate)     # Detects the beginning of notes and calculating the maximum note repetitions in the audio.
    noteIntervals = np.diff(noteOnsets)
    maxNoteReps = np.max(np.bincount(noteIntervals))

    return rhythmicStability, noteDistributionStability, maxNoteReps

# Criteria 3 Definition ========================================================

def analyzeMelodicStructure(audioFile):
    audioData, samplingRate = librosa.load(audioFile)

    pitches, magnitudes = librosa.core.piptrack(y = audioData, sr = samplingRate) # Compute the pitch (fundamental frequency - lowest frequency) over time

    pitchContour = pitches[np.argmax(magnitudes, axis=0)]
    pitchContour = pitchContour.ravel()  # Flatten to a 1D Array.

    melodicIntervals = np.diff(pitchContour)                                    # Melodic intervals are the differences between consecutive pitch values.
    melodicPeaks, _ = find_peaks(pitchContour)
    melodicComplexity = np.std(melodicIntervals)                                # Measures the standard deviation of the intervals between consecutive pitches in the melodic contour.


    melodicStabilityRange = np.max(pitchContour) - np.min(pitchContour)         # Measures the span between the highest and lowest pitches in the melodic contour.
    noteDurations = np.diff(melodicPeaks) / samplingRate
    dynamicVariation = np.std(audioData)

    return melodicPeaks, melodicComplexity, melodicStabilityRange, noteDurations, dynamicVariation

# Data to CSV Implementation ===================================================

for filename in os.listdir(csvCombinedTest):
  totalNotes = 0

  if "random" in filename:
    isSignificant = False
  else:
      isSignificant = True

  if filename.endswith(".wav"):
    filepath = os.path.join(csvCombinedTest, filename)
    audioData, sampleRate = librosa.load(filepath, sr = None)
    inTuneCount = 0

    frequencyEstimatesPYIN = librosa.pyin(audioData, fmin = librosa.note_to_hz('C2'), fmax = librosa.note_to_hz('C7'), fill_na = -1)
    frequencies = frequencyEstimatesPYIN[0]
    magnitudes = frequencyEstimatesPYIN[1]


    for i in range(0, len(frequencies), 5):
      frequency = frequencies[i]
      if frequency > 0:
        closestNoteFrequency = findClosestNoteFrequency(frequency)
        if abs(frequency - closestNoteFrequency) <= noteInTuneTolerance:
            inTuneCount += 1
        totalNotes+=1

    percentageInTune = (inTuneCount / totalNotes) * 100

# End of Criteria 1 ############################################################

    rhythmStability, noteDistributionStability, maxNoteRepetitions = hasRandomNotes(filepath)

# End of Criteria 2 ############################################################

    melodicPeaks, melodicComplexity, melodicStabilityRange, noteDurations, dynamicVariation = analyzeMelodicStructure(filepath)

# End of Criteria 3 ############################################################

    fileData = {
      'Filenumber': fileNumber,
      'Filename': filename,
      '% In-Tune Notes': percentageInTune,
      'Rhythmic Stability': rhythmStability,
      'Note Distribution Stability': noteDistributionStability,
      'Melodic Stability Range': melodicStabilityRange,
      'Dynamic Variation': dynamicVariation,
      'Significant': isSignificant
    }

    fileDataList.append(fileData)
    print(f"File {fileNumber}: Finished appending data for [{filename}]\n")
    fileNumber+=1

csv_columns = [
  'Filenumber', 'Filename', '% In-Tune Notes', 'Rhythmic Stability',
  'Note Distribution Stability',
  'Melodic Stability Range', 'Dynamic Variation', 'Significant'
]

csvFileName = 'combinedTest.csv'

with open(csvFileName, 'w', newline = '') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=csv_columns)

    # Write the header
    writer.writeheader()

    # Write the data for each file
    for data in fileDataList:
        writer.writerow(data)

print(f"Data has been written to {csvFileName}")

File 1: Finished appending data for [Copy of randomMONOwav140.wav]

File 2: Finished appending data for [Copy of randomMONOwav142.wav]

File 3: Finished appending data for [Copy of randomMONOwav145.wav]

File 4: Finished appending data for [Copy of randomMONOwav141.wav]

File 5: Finished appending data for [Copy of randomMONOwav144.wav]

File 6: Finished appending data for [Copy of randomMONOwav148.wav]

File 7: Finished appending data for [Copy of randomMONOwav146.wav]

File 8: Finished appending data for [Copy of randomMONOwav147.wav]

File 9: Finished appending data for [Copy of randomMONOwav143.wav]

File 10: Finished appending data for [Copy of randomMONOwav149.wav]

File 11: Finished appending data for [Copy of randomMONOwav139.wav]

File 12: Finished appending data for [Copy of randomMONOwav137.wav]

File 13: Finished appending data for [Copy of randomMONOwav138.wav]

File 14: Finished appending data for [Copy of randomPOLYwav149.wav]

File 15: Finished appending data for [Copy 